In [1]:
import torch
import pandas as pd
import numpy as np
import os
from pathlib import Path
from dataclasses import dataclass

In [2]:
%pwd

'/home/priyanshu1303d/Projects/Heart_Segmentation/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/priyanshu1303d/Projects/Heart_Segmentation'

In [5]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    images_dir: Path
    labels_dir: Path
    model_metrics_json: Path
    model_save_path : Path
    params_batch_size: int


In [ ]:
from Heart_Segmentation.constants import *
from Heart_Segmentation.utils.common import read_yaml , create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            images_dir=Path(config.images_dir),
            labels_dir=Path(config.labels_dir),
            model_metrics_json=Path(config.model_metrics_json),
            model_save_path = Path(config.model_save_path),
            params_batch_size=self.params.BATCH_SIZE
        )
        return model_evaluation_config

In [8]:
from Heart_Segmentation.constants import *
from Heart_Segmentation.utils.common import read_yaml , create_directories
from Heart_Segmentation import logger
from torch.utils.data import Dataset , DataLoader
import nibabel as nib

In [9]:
# 2. Dataset (Reusing HeartSegmentationDataset from training)
class HeartSegmentationDataset(Dataset):
    def __init__(self, images_dir: str, labels_dir: str):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.image_files = [f for f in os.listdir(images_dir) if f.endswith('.nii.gz')]
        logger.info(f"Found {len(self.image_files)} files in {images_dir}")
        if not self.image_files:
            raise ValueError(f"No .nii.gz files found in {images_dir}")

    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        image_path = os.path.join(self.images_dir, image_file)
        label_path = os.path.join(self.labels_dir, image_file)
        
        image = nib.load(image_path).get_fdata()
        label = nib.load(label_path).get_fdata()
        
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # [1, 128, 128, 64]
        label = torch.tensor(label, dtype=torch.long)  # [128, 128, 64]
        
        return image, label, image_file

In [10]:
from monai.networks.nets import UNet
import json
from Heart_Segmentation.utils.common import save_json

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Load the trained U-Net model
        self.model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=3,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2
        ).to(self.device)
        self.model.load_state_dict(torch.load(self.config.model_save_path))
        self.model.eval()
        
        # DataLoader for evaluation
        dataset = HeartSegmentationDataset(self.config.images_dir, self.config.labels_dir)
        self.dataloader = DataLoader(dataset, batch_size=self.config.params_batch_size, shuffle=False, pin_memory=True)

    def calculate_dice_score(self, pred: torch.Tensor, target: torch.Tensor, smooth=1e-5):
        pred_flat = pred.flatten()
        target_flat = target.flatten()
        intersection = (pred_flat * target_flat).sum()
        return (2. * intersection + smooth) / (pred_flat.sum() + target_flat.sum() + smooth)

    def calculate_iou(self, pred: torch.Tensor, target: torch.Tensor, smooth=1e-5):
        pred_flat = pred.flatten()
        target_flat = target.flatten()
        intersection = (pred_flat * target_flat).sum()
        union = pred_flat.sum() + target_flat.sum() - intersection
        return (intersection + smooth) / (union + smooth)

    def save_metrics_to_json(self, metrics: dict):
        """Save evaluation metrics to a JSON file."""
        json_path = self.config.model_metrics_json
        try:
            with open(json_path, 'w') as f:
                json.dump(metrics, f, indent=4)
            logger.info(f"Metrics saved to {json_path}")
        except Exception as e:
            logger.error(f"Failed to save metrics to {json_path}: {str(e)}")
            raise e

    def evaluate(self):
        """Run evaluation, compute metrics, and save them to JSON."""
        logger.info("-------------Started Evaluation----------")
        dice_scores = []
        iou_scores = []
        accuracies = []
        confidences = []

        with torch.no_grad():
            for images, labels, _ in self.dataloader:
                images, labels = images.to(self.device), labels.to(self.device)
                
                outputs = self.model(images)
                probs = torch.softmax(outputs, dim=1)
                preds = torch.argmax(probs, dim=1)
                
                confidence = torch.max(probs, dim=1)[0].mean().cpu().item()
                confidences.append(confidence)

                for pred, label in zip(preds, labels):
                    pred_binary = (pred > 0).float()
                    label_binary = (label > 0).float()
                    
                    dice = self.calculate_dice_score(pred_binary, label_binary)
                    iou = self.calculate_iou(pred_binary, label_binary)
                    accuracy = (pred == label).float().mean().cpu().item()
                    
                    dice_scores.append(dice.cpu().item())
                    iou_scores.append(iou.cpu().item())
                    accuracies.append(accuracy)

        # Average metrics
        avg_dice = np.mean(dice_scores)
        avg_iou = np.mean(iou_scores)
        avg_accuracy = np.mean(accuracies) * 100
        avg_confidence = np.mean(confidences) * 100

        # Log results
        logger.info(f"Evaluation Results:")
        logger.info(f"Average Dice Score: {avg_dice:.4f}")
        logger.info(f"Average IoU Score: {avg_iou:.4f}")
        logger.info(f"Average Accuracy: {avg_accuracy:.2f}%")
        logger.info(f"Average Confidence Score: {avg_confidence:.2f}%")

       # Save metrics to JSON using the imported save_json function
        metrics = {
            "average_dice_score": avg_dice,
            "average_iou_score": avg_iou,
            "average_accuracy_percent": avg_accuracy,
            "average_confidence_score_percent": avg_confidence
        }
        save_json(path=self.config.model_metrics_json, data=metrics)

        return avg_dice, avg_iou, avg_accuracy, avg_confidence

In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_model_evaluation_config()
    model_eval = ModelEvaluation(eval_config)
    model_eval.evaluate()
except Exception as e:
    logger.error(f"Error during evaluation: {str(e)}")
    raise e

[2025-03-26 23:20:32,571: INFO: common : yaml file: config/config.yaml loaded successfully]
[2025-03-26 23:20:32,575: INFO: common : yaml file: params.yaml loaded successfully]
[2025-03-26 23:20:32,577: INFO: common : created directory at: artifacts]


[2025-03-26 23:20:32,578: INFO: common : created directory at: artifacts/model_evaluation]
[2025-03-26 23:20:32,812: INFO: 3417093127 : Found 20 files in artifacts/data_preprocessing/preprocessed]
[2025-03-26 23:20:32,813: INFO: 1401051403 : -------------Started Evaluation----------]
[2025-03-26 23:20:34,893: INFO: 1401051403 : Evaluation Results:]
[2025-03-26 23:20:34,894: INFO: 1401051403 : Average Dice Score: 0.3684]
[2025-03-26 23:20:34,895: INFO: 1401051403 : Average IoU Score: 0.2287]
[2025-03-26 23:20:34,895: INFO: 1401051403 : Average Accuracy: 99.64%]
[2025-03-26 23:20:34,896: INFO: 1401051403 : Average Confidence Score: 95.92%]
[2025-03-26 23:20:34,897: INFO: common : json file saved at : artifacts/model_evaluation/dict.json]
